In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np

ttype = torch.cuda.DoubleTensor if torch.cuda.is_available() else torch.DoubleTensor
torch.set_default_tensor_type(ttype)

from lqg_env import LQG
from util import simulate, seiler_state_space, StateSpace, train_model, eval_model
from controllers import RNNController, OptimalController

In [3]:
import matplotlib.pyplot as plt

n_epochs = 150
num_inits = 200
n_steps = 100
batch_size = 128
rnn_layers = 3
dim_hidden = 10
learning_rate = 0.01

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ss, Q, R, = seiler_state_space()

controller = RNNController(1, 1, dim_h=dim_hidden, layers=rnn_layers).to(device)
env = LQG(ss, Q, R, batch_size=batch_size)
env_x = LQG(ss, Q, R, batch_size=batch_size, observe_x=True)

opt = torch.optim.Adam(controller.parameters(), lr=learning_rate)

best = (np.inf, None)

optimal_controller = OptimalController(ss, Q, R)
loss = simulate(optimal_controller, env_x, n_steps)
print("optimal:", loss.detach().cpu().numpy().item())

optimal: 52195.19897313523


In [26]:
for i in range(1, 11):
    controller.load_state_dict(torch.load(f'models/rnn_best_initial_{i}'))
    loss = simulate(controller, env, n_steps)
    print("i:", np.log10(loss.detach().cpu().numpy().item()))

i: 12.147936287853012
i: 12.085002750858145
i: 12.098255151675502
i: 12.361476036478155
i: 11.994642748667932
i: 12.017809383328332
i: 11.891365516277045
i: 11.971245720769641
i: 12.011292277276297
i: 12.372666196106685
